In [34]:
import os
from pymongo import MongoClient
from dotenv import load_dotenv

load_dotenv()

MONGO_URL = os.getenv("CLIENT_URL")
DB_NAME = "migracion"
COLLECTION_NAME = "documentos"

In [2]:
from langchain_openai import OpenAIEmbeddings

client = MongoClient(MONGO_URL)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

# Convertir los textos en embeddings
embeddings_model = OpenAIEmbeddings()

In [3]:
from openai import OpenAI

client = OpenAI()
embeddings_name = "text-embedding-3-small"

def generate_embeddings(text: str) -> list[float]:
    response = client.embeddings.create(
        input=text,
        model=embeddings_name
    )
    return response.data[0].embedding

In [31]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Cargar el PDF
# pdf_path = "files/BOE-A-2009-17242.pdf"
pdf_path = "files/BOE-A-2022-4978-consolidado.pdf"
# pdf_path = "files/folleto_PI_espanol.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Dividir el texto en fragmentos manejables
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [17]:
texts[0]

Document(metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2023-10-13T10:02:28+02:00', 'author': 'Sergio Jiménez Fernández', 'moddate': '2023-10-13T10:02:28+02:00', 'source': 'files/folleto_PI_espanol.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='INFORMACIÓN PARA LOS \nSOLICITANTES DE PROTECCIÓN \nINTERNACIONAL EN ESPAÑA: \nDERECHO DE ASILO  \nY PROTECCIÓN SUBSIDIARIA \n \n \n1 \n \n   \n \n \n• A los refugiados , es decir , a las personas extranjeras de nacionalidad no comunitaria o \napátridas que tengan un temor fundado a ser perseguidas en su país por motivos de raza, \nreligión, nacionalidad, opiniones políticas, pertenencia a determinado grupo social, de \ngénero u orientaci ón sexual y que no puedan o no quieran por ese motivo recibir la \nprotección de las autoridades de su país. A los refugiados se les reconoce el derecho de \nasilo. \n \n• A los extranjeros que no cumplan los re

In [32]:
# Guardar en MongoDB
for text in texts:
    embedding = generate_embeddings(text.page_content)
    collection.insert_one({
        "text": text.page_content,
        "embedding": embedding,
        "metadata": text.metadata
    })

print("Embeddings almacenados en MongoDB correctamente.")

Embeddings almacenados en MongoDB correctamente.


In [ ]:

embeddings = embeddings_model.embed_documents([t.page_content for t in texts])

# Guardar en MongoDB
for i, (text, embedding) in enumerate(zip(texts, embeddings)):
    collection.insert_one({
        "text": text.page_content,
        "embedding": embedding,
        "metadata": text.metadata
    })

print("Embeddings almacenados en MongoDB correctamente.")

Embeddings almacenados en MongoDB correctamente.


In [ ]:
from src.db.mongo import MongoRepository

mongo_repo = MongoRepository()

In [4]:
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch

vectore_store = MongoDBAtlasVectorSearch.from_connection_string(
            connection_string=MONGO_URL or "",
            namespace=f"{DB_NAME}.{COLLECTION_NAME}",
            embedding=embeddings_model,
        )

In [5]:
result = vectore_store.similarity_search("Cómo puedo realizar un asilo político?",k=7)
result

[Document(metadata={'_id': '67b24b3066b11d0008239a47', 'metadata': {'producer': 'Antenna House PDF Output Library 6.6.1477 (Linux64)', 'creator': 'AH XSL Formatter V6.6 MR7 for Linux64 : 6.6.9.39847 (2019-07-29T09:58+09)', 'creationdate': '2025-02-13T08:28:50+01:00', 'title': 'Real Decreto 220/2022, de 29 de marzo, por el que se aprueba el Reglamento por el que se regula el sistema de acogida en materia de protección internacional.', 'author': 'Agencia Estatal Boletín Oficial del Estado', 'subject': 'BOE-A-2022-4978 actualizado a 30 de marzo de 2022', 'keywords': 'BOE-A-2022-4978; BOE; Legislación consolidada; Agencia Estatal Boletín Oficial del Estado', 'moddate': '2025-02-13T08:28:50+01:00', 'trapped': '/False', 'source': 'files/BOE-A-2022-4978-consolidado.pdf', 'total_pages': 28, 'page': 21, 'page_label': '22'}}, page_content='2. Cuando se produzca demora en las prestaciones o servicios por parte de la entidad \nautorizada, el órgano de concertación podrá conceder una ampliación del

In [33]:
client = MongoClient(MONGO_URL)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

print(collection.count_documents({})) 

240


## Resultados del BOE

In [54]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# pdf_path = "files/BOE-2024-11-01.pdf"
# pdf_path = "files/BOE-2024-11-15.pdf"
pdf_path = "files/BOE-2024-12-02.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()


In [48]:
from pydantic import BaseModel, Field

class Resolucion(BaseModel):
    nie: str = Field(description="Número de Identificación de Extranjero", example="X1234567A")
    expediente: int = Field(description="Número de expediente", example=123456)
    tipo_archivo: str = Field(description="Tipo de archivo", example="asilo")
    Num_resolucion: int = Field(description="Número de resolución del BOE", example=260)
    fecha_resolucion: str = Field(description="Fecha de resolución", example="2024-01-11")
    pagina: int = Field(description="Página de la resolución", example=1)

class Resoluciones(BaseModel):
    resoluciones: list[Resolucion]

In [63]:
Resolucion.model_json_schema()

{'properties': {'nie': {'description': 'Número de Identificación de Extranjero',
   'example': 'X1234567A',
   'title': 'Nie',
   'type': 'string'},
  'expediente': {'description': 'Número de expediente',
   'example': 123456,
   'title': 'Expediente',
   'type': 'integer'},
  'tipo_archivo': {'description': 'Tipo de archivo',
   'example': 'asilo',
   'title': 'Tipo Archivo',
   'type': 'string'},
  'Num_resolucion': {'description': 'Número de resolución del BOE',
   'example': 260,
   'title': 'Num Resolucion',
   'type': 'integer'},
  'fecha_resolucion': {'description': 'Fecha de resolución',
   'example': '2024-01-11',
   'title': 'Fecha Resolucion',
   'type': 'string'},
  'pagina': {'description': 'Página de la resolución',
   'example': 1,
   'title': 'Pagina',
   'type': 'integer'}},
 'required': ['nie',
  'expediente',
  'tipo_archivo',
  'Num_resolucion',
  'fecha_resolucion',
  'pagina'],
 'title': 'Resolucion',
 'type': 'object'}

In [55]:
from langchain_openai import ChatOpenAI


prompt = """
    You are an expert in extract information from legal documents. 
    You have to extract the following information from the document:
    
    - NIE: X1234567A
    - Expediente: 123456
    - Tipo de archivo: asilo
    - Número de resolución: 260
    - Fecha de resolución: 2024-01-11
    - Página: 1
    """


model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = model.with_structured_output(Resoluciones)

In [21]:
response = structured_llm.invoke(documents[0].page_content)

In [56]:
resoluciones = []
for document in documents:
    response = structured_llm.invoke(document.page_content)
    try:
        resoluciones.extend(response.resoluciones)
    except:
        print(f"No se ha podido extraer la información de la resolución {response}")

In [57]:
len(resoluciones)

1715

In [58]:
from pymongo import MongoClient
import os

from dotenv import load_dotenv

load_dotenv()

client_db = MongoClient(MONGO_URL)
db = client_db[DB_NAME]
collection = db["resoluciones"]

In [59]:
import json


resoluciones_dict = [json.loads(resolucion.model_dump_json()) for resolucion in resoluciones]

In [60]:
collection.insert_many(resoluciones_dict)

InsertManyResult([ObjectId('67b478394a7ebd69a888bdad'), ObjectId('67b478394a7ebd69a888bdae'), ObjectId('67b478394a7ebd69a888bdaf'), ObjectId('67b478394a7ebd69a888bdb0'), ObjectId('67b478394a7ebd69a888bdb1'), ObjectId('67b478394a7ebd69a888bdb2'), ObjectId('67b478394a7ebd69a888bdb3'), ObjectId('67b478394a7ebd69a888bdb4'), ObjectId('67b478394a7ebd69a888bdb5'), ObjectId('67b478394a7ebd69a888bdb6'), ObjectId('67b478394a7ebd69a888bdb7'), ObjectId('67b478394a7ebd69a888bdb8'), ObjectId('67b478394a7ebd69a888bdb9'), ObjectId('67b478394a7ebd69a888bdba'), ObjectId('67b478394a7ebd69a888bdbb'), ObjectId('67b478394a7ebd69a888bdbc'), ObjectId('67b478394a7ebd69a888bdbd'), ObjectId('67b478394a7ebd69a888bdbe'), ObjectId('67b478394a7ebd69a888bdbf'), ObjectId('67b478394a7ebd69a888bdc0'), ObjectId('67b478394a7ebd69a888bdc1'), ObjectId('67b478394a7ebd69a888bdc2'), ObjectId('67b478394a7ebd69a888bdc3'), ObjectId('67b478394a7ebd69a888bdc4'), ObjectId('67b478394a7ebd69a888bdc5'), ObjectId('67b478394a7ebd69a888bd

In [62]:
print(collection.count_documents({})) 

5975
